<a href="https://colab.research.google.com/github/RodrigoAgronomia/workshop_esalq/blob/master/python/24_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.4 Data augmentation

In [0]:
%%capture
!pip install Augmentor
%cd /content
!git clone --depth 1 https://github.com/RodrigoAgronomia/workshop_esalq.git
%cd /content/workshop_esalq
!git pull
%cd /content/workshop_esalq/python

In [0]:
#Importação das bibliotecas necessárias para rodar o código
%matplotlib inline

import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import re

In [0]:
import Augmentor

In [0]:
#Define o mapa de cores para geração da máscara
def colorir(cat):
    h, w = cat.shape[:2]
    msk = np.zeros((h,w,3), dtype = 'uint8')
    msk[cat == 1] = [0,0,255]
    msk[cat == 2] = [128,0,0]
    msk[cat == 3] = [255,0,0]
    msk[cat == 4] = [0,128,0]
    msk[cat == 5] = [0,255,0]
    return(msk)

In [0]:
def get_mask(imgt):
    #Split Image in 3 channels
    blue,green,red = cv2.split(imgt)
    #Normalize values
    green = np.divide(green,255)
    red = np.divide(red,255)
    blue = np.divide(blue,255)

    #Sum of normalized values
    norm = red+green+blue

    r = np.divide(red, norm, out=np.zeros_like(red), where=norm!=0)
    g = np.divide(green, norm, out=np.zeros_like(green), where=norm!=0)
    b = np.divide(blue, norm, out=np.zeros_like(blue), where=norm!=0)
    
    #ExG - ExR
    midx = ((2*g) - r - b) - ((1.4*r) - g)

    #Binarize image
    midx[midx<0] = 0
    midx[midx>0] = 1
    imager = (midx*255).astype('uint8')
    return(imager)

In [0]:
#Definição das pastas necessárias para rodar o script
img_dir = '../data/orig/'
s_dir = '../data/seg/imgs_augmented/'
anotacao = '../data/anotations.csv'

In [10]:
#Apresenta as imagens dentro do diretório com o respectivo formato da imagem
img_list = os.listdir(img_dir)
img_list = [v for v in img_list if v.endswith('.JPG')]
print(img_list)

['DJI_0411.JPG', 'DJI_0417.JPG', 'DJI_0400.JPG', 'DJI_0393.JPG', 'DJI_0335.JPG', 'DJI_0339.JPG', 'DJI_0337.JPG', 'DJI_0404.JPG', 'DJI_0353.JPG', 'DJI_0419.JPG', 'DJI_0392.JPG', 'DJI_0407.JPG', 'DJI_0399.JPG', 'DJI_0348.JPG', 'DJI_0346.JPG', 'DJI_0406.JPG', 'DJI_0412.JPG', 'DJI_0394.JPG', 'DJI_0413.JPG', 'DJI_0341.JPG', 'DJI_0356.JPG', 'DJI_0350.JPG', 'DJI_0361.JPG', 'DJI_0352.JPG', 'DJI_0421.JPG', 'DJI_0408.JPG', 'DJI_0418.JPG', 'DJI_0351.JPG', 'DJI_0395.JPG', 'DJI_0396.JPG', 'DJI_0355.JPG', 'DJI_0344.JPG', 'DJI_0345.JPG', 'DJI_0360.JPG', 'DJI_0340.JPG', 'DJI_0420.JPG', 'DJI_0332.JPG', 'DJI_0414.JPG', 'DJI_0342.JPG', 'DJI_0397.JPG', 'DJI_0410.JPG', 'DJI_0398.JPG', 'DJI_0358.JPG', 'DJI_0357.JPG', 'DJI_0402.JPG', 'DJI_0354.JPG', 'DJI_0338.JPG', 'DJI_0409.JPG', 'DJI_0405.JPG', 'DJI_0349.JPG', 'DJI_0336.JPG', 'DJI_0403.JPG', 'DJI_0359.JPG', 'DJI_0343.JPG', 'DJI_0334.JPG', 'DJI_0347.JPG', 'DJI_0415.JPG', 'DJI_0333.JPG', 'DJI_0401.JPG', 'DJI_0416.JPG']


In [12]:
n_classes = 6
images = []

csv_file = pd.read_csv(anotacao,index_col = False) #Faz a leitura do arquivo .csv

for img in img_list:
    lista = csv_file[csv_file.filename == img] #Recebe os dados encontrados dentro do csv para a imagem em questão
    tamanho = lista.shape
    if tamanho[0] != 0:
        img_name = os.path.join(img_dir, img)
        print(img_name) 
        
        imagem = cv2.imread(img_name) #Carrega a imagem em questão

        msk = np.zeros(imagem.shape[:2], dtype = 'uint8') #Carrega as informações do tamanho da imagem
        # Cria uma mascara auxiliar para o desenho dos circulos ao redor das plantas:
        msk0 = msk.copy()
        msk1 = msk.copy()
        
        # Importa a mascara do indice de vegetacao:
        mskk = get_mask(imagem)
        mskk = (mskk > 100).astype('uint8') 

    
        for i in range(tamanho[0]):
            line = lista.iloc[i,:] #Recebe todas as marcações realizadas dentro daquela imagem
            region_shape = line.region_shape_attributes #Informa a posição onde o ponto se encontra
            region_attributes = (line.region_attributes) #Informa a classe do ponto
            region_attributes = re.findall('"([^"]*)"', region_attributes) #Faz a separação para encontrar a qual classe pertence o ponto marcado
            coords = [float(s) for s in re.findall(r'-?\d+\.?\d*', region_shape)] #Encontrando valores de x e y
            cx = int(coords[0]) #Coordenadas no eixo X
            cy = int(coords[1]) #Coordenadas no eixo y

            #Define as classes aqui para realizar o desenho da máscara      
            if region_attributes[1] == 'Corn':
                cv2.circle(msk0, (cx,cy), 80, 2, -1)
                cv2.circle(msk1, (cx,cy), 16, 3, -1)
                
                
            if region_attributes[1] == 'Soybean':
                cv2.circle(msk0, (cx,cy), 40, 4, -1)
                cv2.circle(msk1, (cx,cy), 12, 5, -1)
        
        #Faz a combinação das mascaras na máscara principal
        msk[(msk0==2)&(mskk==1)] = 2
        msk[(msk0==4)&(mskk==1)] = 4
        msk[(msk1==3)] = 3
        msk[(msk1==5)] = 5
        msk[(msk==0)&(mskk==1)] = 1
        
        im_l = [imagem]
        for n in range(n_classes):
            mskn = msk == n
            mskn = 255*mskn.astype('uint8')
            im_l.append(mskn)
        images.append(im_l)
        
    else:
        print('A imagem'+' '+img+' '+'não possui anotação no .csv apresentado') #Apresenta uma mensagem caso a imagem lida não apresente informações dentro do .csv

../data/orig/DJI_0411.JPG
../data/orig/DJI_0417.JPG
../data/orig/DJI_0400.JPG
../data/orig/DJI_0393.JPG
../data/orig/DJI_0335.JPG
../data/orig/DJI_0339.JPG
../data/orig/DJI_0337.JPG
../data/orig/DJI_0404.JPG
../data/orig/DJI_0353.JPG
../data/orig/DJI_0419.JPG
../data/orig/DJI_0392.JPG
../data/orig/DJI_0407.JPG
../data/orig/DJI_0399.JPG
../data/orig/DJI_0348.JPG
../data/orig/DJI_0346.JPG
../data/orig/DJI_0406.JPG
../data/orig/DJI_0412.JPG
../data/orig/DJI_0394.JPG
../data/orig/DJI_0413.JPG
../data/orig/DJI_0341.JPG
../data/orig/DJI_0356.JPG
../data/orig/DJI_0350.JPG
../data/orig/DJI_0361.JPG
../data/orig/DJI_0352.JPG
../data/orig/DJI_0421.JPG
../data/orig/DJI_0408.JPG
../data/orig/DJI_0418.JPG
../data/orig/DJI_0351.JPG
../data/orig/DJI_0395.JPG
../data/orig/DJI_0396.JPG
../data/orig/DJI_0355.JPG
../data/orig/DJI_0344.JPG
../data/orig/DJI_0345.JPG
../data/orig/DJI_0360.JPG
../data/orig/DJI_0340.JPG
../data/orig/DJI_0420.JPG
../data/orig/DJI_0332.JPG
../data/orig/DJI_0414.JPG
../data/orig

In [0]:
p = Augmentor.DataPipeline(images)

p.crop_by_size(1, 256, 256, centre = False)
p.zoom_random(probability=0.5, percentage_area=0.8)
p.rotate90(probability=0.5)
p.rotate270(probability=0.5)
p.rotate(probability=0.5, max_left_rotation=25, max_right_rotation=25)
p.flip_left_right(probability=0.5)
p.flip_top_bottom(probability=0.5)
p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=8)
p.histogram_equalisation(0.25)
p.resize(probability=1.0, width=256, height=256)

In [0]:
augmented_images = p.sample(100)

In [0]:
try:
    os.mkdir(s_dir)
except:
    pass
for i, img in enumerate(augmented_images):
    img_name = 'RGB_{0:04d}_img.jpg'.format(i + 1)
    msk_name = 'RGB_{0:04d}_msk.png'.format(i + 1)
    msk_col_name = 'RGB_{0:04d}_col.png'.format(i + 1)
    msk = np.array(img[1:]).argmax(0)
    if np.sum((msk == 3) + (msk == 5)) > 10:
        cv2.imwrite(os.path.join(s_dir, img_name), img[0])
        cv2.imwrite(os.path.join(s_dir, msk_name), msk)    
        cv2.imwrite(os.path.join(s_dir, msk_col_name), colorir(msk))    
